In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, ensemble, linear_model, neural_network, metrics, inspection
from soyviz import plot_los_tres, plot_estres
import matplotlib.pyplot as plt
from combat.pycombat import pycombat
from pickle import load, dump

In [2]:
venancio = pd.read_parquet('../expresion/venancio_bias_corrected.parquet')
venancio

Gene,Glyma.01G000100,Glyma.01G000137,Glyma.01G000174,Glyma.01G000211,Glyma.01G000248,Glyma.01G000285,Glyma.01G000322,Glyma.01G000359,Glyma.01G000400,Glyma.01G000600,...,Glyma.U044501,Glyma.U044700,Glyma.U044800,Glyma.U045000,Glyma.U045100,Glyma.U045402,Glyma.U045502,Glyma.U045602,Glyma.U045702,Glyma.U045802
SAMD00025071,16,4,11,2,5,12,11,8,70,102,...,0,0,0,0,0,1,0,0,1,0
SAMD00025072,9,2,3,0,0,2,6,1,17,31,...,0,0,0,0,0,1,0,0,0,0
SAMD00025073,16,4,10,0,13,11,9,7,76,102,...,0,0,0,0,0,4,0,3,1,0
SAMD00133785,0,0,1,6,0,0,69,0,209,101,...,0,0,0,0,0,0,0,2,0,67
SAMD00133786,0,0,0,0,0,2,44,0,143,190,...,0,0,0,0,0,3,0,0,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN14277281,47,4,6,8,4,6,15,20,752,517,...,0,0,0,0,0,1,0,15,1,1
SAMN14277282,68,6,10,16,3,7,25,16,898,503,...,0,4,0,0,3,10,0,7,0,4
SAMN14277283,270,33,49,60,34,25,70,68,826,622,...,0,0,0,0,0,15,0,22,1,3
SAMN14277284,176,37,31,66,30,24,83,29,1182,764,...,0,0,0,0,0,32,0,11,7,11


In [6]:
annot = pd.read_csv('../anotacion/gemini_annot.csv', index_col=0)
annot = annot[annot['tejido'].notna() & annot['estres'].notna()]
# annot = annot.loc[(annot['tejido'] == 'leaf') | (annot['tejido'] == 'root') | (annot['tejido'] == 'seed') | (annot['tejido'] == 'stem')]
annot

,bioproject,tejido,estres,tratamiento
biosample,,,,
SAMD00235524,PRJDB10183,seedling,control,control
SAMD00235525,PRJDB10183,seedling,control,control
SAMD00235526,PRJDB10183,seedling,control,control
SAMD00235527,PRJDB10183,seedling,control,control
SAMD00235528,PRJDB10183,seedling,control,control
...,...,...,...,...
SAMN36760719,PRJNA999924,seed,cold,treatment
SAMN36760720,PRJNA999924,seed,cold,treatment
SAMN36760721,PRJNA999924,seed,cold,treatment


## Preprocesado

In [7]:
venancio, annot = venancio.align(annot, join='inner', axis=0)
venancio

Gene,Glyma.01G000100,Glyma.01G000137,Glyma.01G000174,Glyma.01G000211,Glyma.01G000248,Glyma.01G000285,Glyma.01G000322,Glyma.01G000359,Glyma.01G000400,Glyma.01G000600,...,Glyma.U044501,Glyma.U044700,Glyma.U044800,Glyma.U045000,Glyma.U045100,Glyma.U045402,Glyma.U045502,Glyma.U045602,Glyma.U045702,Glyma.U045802
SAMD00133785,0,0,1,6,0,0,69,0,209,101,...,0,0,0,0,0,0,0,2,0,67
SAMD00133786,0,0,0,0,0,2,44,0,143,190,...,0,0,0,0,0,3,0,0,3,12
SAMD00216748,56,22,35,28,65,47,83,17,793,606,...,0,0,0,3,0,18,0,0,0,16
SAMD00216750,43,27,21,14,33,29,58,12,462,476,...,0,0,0,0,0,12,0,3,1,27
SAMD00216751,66,18,27,27,49,28,38,9,240,234,...,21,0,0,0,0,9,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN14277281,47,4,6,8,4,6,15,20,752,517,...,0,0,0,0,0,1,0,15,1,1
SAMN14277282,68,6,10,16,3,7,25,16,898,503,...,0,4,0,0,3,10,0,7,0,4
SAMN14277283,270,33,49,60,34,25,70,68,826,622,...,0,0,0,0,0,15,0,22,1,3
SAMN14277284,176,37,31,66,30,24,83,29,1182,764,...,0,0,0,0,0,32,0,11,7,11


In [8]:
consamples = annot['bioproject'].map(annot['bioproject'].value_counts() > 2)
convarianza = venancio.var() > 0.01

filtrado = venancio.loc[consamples, convarianza]

filtrado

Gene,Glyma.01G000100,Glyma.01G000137,Glyma.01G000174,Glyma.01G000211,Glyma.01G000248,Glyma.01G000285,Glyma.01G000322,Glyma.01G000359,Glyma.01G000400,Glyma.01G000600,...,Glyma.U044501,Glyma.U044700,Glyma.U044800,Glyma.U045000,Glyma.U045100,Glyma.U045402,Glyma.U045502,Glyma.U045602,Glyma.U045702,Glyma.U045802
SAMD00133785,0,0,1,6,0,0,69,0,209,101,...,0,0,0,0,0,0,0,2,0,67
SAMD00133786,0,0,0,0,0,2,44,0,143,190,...,0,0,0,0,0,3,0,0,3,12
SAMD00216748,56,22,35,28,65,47,83,17,793,606,...,0,0,0,3,0,18,0,0,0,16
SAMD00216750,43,27,21,14,33,29,58,12,462,476,...,0,0,0,0,0,12,0,3,1,27
SAMD00216751,66,18,27,27,49,28,38,9,240,234,...,21,0,0,0,0,9,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN14277281,47,4,6,8,4,6,15,20,752,517,...,0,0,0,0,0,1,0,15,1,1
SAMN14277282,68,6,10,16,3,7,25,16,898,503,...,0,4,0,0,3,10,0,7,0,4
SAMN14277283,270,33,49,60,34,25,70,68,826,622,...,0,0,0,0,0,15,0,22,1,3
SAMN14277284,176,37,31,66,30,24,83,29,1182,764,...,0,0,0,0,0,32,0,11,7,11


In [ ]:
filtrado, annot = filtrado.align(annot, join='inner', axis=0)

combat = pycombat(filtrado.T, annot['bioproject']).T
combat

Found 178 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.


/home/godo/.local/lib/python3.10/site-packages/combat/pycombat.py:159: RuntimeWarning: divide by zero encountered in divide
  np.absolute(d_new-d_old)/d_old))  # maximum difference between new and old estimate
/home/godo/.local/lib/python3.10/site-packages/combat/pycombat.py:158: RuntimeWarning: divide by zero encountered in divide
  change = max(np.amax(np.absolute(g_new-np.asarray(g_old))/np.asarray(g_old)), np.amax(


Adjusting the Data


In [ ]:
X, y = combat.align(annot, join='inner', axis=0)
X

In [ ]:
plot_los_tres(X, y)
plt.suptitle('Muestras según estrés')

In [9]:
y = y['estres']

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)
X_train

## Alternativos
### Random forest

In [21]:
random_forest = ensemble.RandomForestClassifier()
params = {
    'n_estimators': np.arange(100, 1000, 100),
    'max_depth': np.arange(20, 50, 10),
    'min_samples_split': np.arange(2, 15, 4),
    'min_samples_leaf': np.arange(2, 15, 4),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}

In [22]:
try:
    with open('forest_4.pkl', 'rb') as f:
        gradient = load(f)
except:
    search = model_selection.RandomizedSearchCV(random_forest, param_distributions=params, n_jobs=-1, scoring='roc_auc_ovr', verbose=3)
    search.fit(X_train, y_train)

    gradient = search.best_estimator_
    with open('forest_4.pkl', 'wb') as f:
        dump(gradient, f, protocol=5)

In [ ]:
forest = search.best_estimator_
forest.get_params()

In [ ]:
forest.score(X_test, y_test)

In [ ]:
metrics.f1_score(y_test, forest.predict(X_test), average='macro')

In [ ]:
importances = pd.DataFrame({'importance': np.flip(np.sort(forest.feature_importances_))}, index=X_train.columns[np.flip(forest.feature_importances_.argsort())])
plt.semilogx(importances['importance'].to_list(), '.')
plt.title('Importancias de features ordenadas (Random Forest)')

In [27]:
importances.to_csv('importancias_4_rf.csv')

### Gradient boosting

In [28]:
gbr = ensemble.GradientBoostingClassifier()
params = {
    "loss": ['log_loss'],
    "learning_rate": np.arange(0, 10, 1),
    "n_estimators": np.arange(1, 200, 1),
    "max_depth": np.arange(1, 50, 1),
    "max_features": ['sqrt', 'log2']
}

In [ ]:
try:
    with open('gradient_4.pkl', 'rb') as f:
        gradient = load(f)
except:
    search = model_selection.RandomizedSearchCV(gbr, param_distributions=params, n_jobs=-1, scoring='roc_auc_ovr', verbose=3)
    search.fit(X_train, y_train)

    gradient = search.best_estimator_
    with open('gradient_4.pkl', 'wb') as f:
        dump(gradient, f, protocol=5)

In [ ]:
gradient.score(X_test, y_test)

In [ ]:
metrics.f1_score(y_test, gradient.predict(X_test), average='macro')

In [ ]:
importances = pd.DataFrame({'importance': np.flip(np.sort(gradient.feature_importances_))}, index=X_train.columns[np.flip(gradient.feature_importances_.argsort())])
plt.semilogx(importances['importance'].to_list(), '.')
plt.title('Importancias de features ordenadas (Gradient boosted trees)')

In [33]:
importances.to_csv('importancias_4_gbr.csv')

## Tradicionales
### DESeq2

In [34]:
from pydeseq2.ds import DeseqStats, DefaultInference, DeseqDataSet
import dash_bio

In [ ]:
fil, ann = filtrado.align(y, join='inner', axis=0)
fil

In [36]:
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=fil,
    metadata=pd.DataFrame({'biotico': ann}),
    design_factors="biotico",  # compare samples based on the "condition"
    refit_cooks=True,
    inference=inference,
)

In [ ]:
dds.deseq2()

In [ ]:
soy_stat = DeseqStats(dds, alpha=0.05, cooks_filter=True, independent_filter=True)
soy_stat.summary()

In [ ]:
np.log(soy_stat.p_values)

In [ ]:
plt.scatter(soy_stat.LFC['tratamiento_treatment_vs_control'], -np.log(soy_stat.p_values), alpha=0.3)
plt.ylabel('-log(p)')
plt.xlabel('Log-fold change')
plt.hlines(18, xmin=-5, xmax=5, linestyles='dashed')
plt.vlines([-0.8, 0.8], ymin=-1, ymax=100, linestyles='dashed')
# plt.xlim(-3, 3)
# plt.ylim(0, 50)
plt.title('Volcano plot PRJNA706999')

In [ ]:
res = soy_stat.results_df[soy_stat.results_df['pvalue'].notna()].reset_index()
res['SNP'] = '1'
dash_bio.VolcanoPlot(dataframe=res.rename({
    'pvalue': 'P',
    'Gene': 'GENE',
    'log2FoldChange': 'EFFECTSIZE'
    # GENE
}, axis=1))